# Libraries

In [2]:
from pathlib import Path
from metadata import ImageDataset, patient
import utils
import numpy as np
import pandas as pd

In [3]:
notebooks_path = Path.cwd()
repo_path = notebooks_path.parent
print(f'The current directory is: {notebooks_path}')

The current directory is: /home/ricardino/Documents/MAIA/tercer_semestre/MISA/final_project/MISA_FINAL_PROJECT/notebooks


# MAIN

We now have the propagated labels, as well as a metric for the similarity between the registered images.<br>
We can now build several versions of the atlas.
We start with the most common, the **probabilistic atlas**.

## Probabilistic atlas

- All atlases (labels) are summed up and divided by the number of images. This is basically weighted voting with the same weight for all images (1/n).

In [60]:
#For each patient in the validation, we will accumulate the labels, summing them.
im_data_val = ImageDataset(set_name='Validation')
im_data_train = ImageDataset(set_name='Training')
#dataframe to store probabilistic atlas
df_maxMean = None
for id_val in im_data_val.IDs:
    #instantiate patient
    pat_val = patient(id_val, im_data_val)
    #accumulated label array
    accumulated_label = np.zeros((4,) + pat_val.im(format='np').shape)
    for id_train in im_data_train.IDs:
        moved_label_path = repo_path / 'data'/'voxelmorph'/f'moved_labels_{id_train}_to_{id_val}.nii.gz'
        moved_label = utils.getArrayfromPath(moved_label_path)
        #accumulate per tissue
        for tissue in range(1,4):
            accumulated_label[tissue] += (moved_label==tissue)
    #divide by the number of labels
    accumulated_label /= im_data_train.len

    #Now we can take the argmax of the accumulated label to get the final mean atlas label
    maxMean_label = np.argmax(accumulated_label, axis=0)
    #get gorund truth
    groundT = pat_val.labels(format='np')
    #compute the dice score
    dice = [utils.dice_score(groundT==tissue, maxMean_label==tissue) for tissue in range(1,4)]
    #save the dice score in a dataframe. The first column is the id_val, then CSF, GM, WM
    df_dice = pd.DataFrame([[id_val] + dice], columns=['id_val', 'CSF', 'GM', 'WM'])
    #concatenate in df_maxM
    df_maxMean = pd.concat([df_maxMean, df_dice], axis=0)
#save as csv
df_maxMean.to_csv(repo_path / 'data'/'results'/f'maxMean_dice.csv', index=False)

## Weighted atlas

- Similar to the previous one, but the weights are not the same for all images. The weights are the similarity metric between the registered images and the target (validation) image.

In [46]:
#Define image datasets
im_data_val = ImageDataset(set_name='Validation')
im_data_train = ImageDataset(set_name='Training')
#dataframe to store probabilistic atlas
df_weightedAtlas = None

for id_val in im_data_val.IDs:
    #instantiate patient
    pat_val = patient(id_val, im_data_val)
    #array to store the weighted atlas
    weighted_atlas = np.zeros((4,) + pat_val.im(format='np').shape)

    #Compute the sum of similarity metrics for this validation patient
    df_mostSimilar = pd.read_csv(repo_path / 'data'/'results'/'most_similar'/ f'most_similar_{id_val}.csv')
    #get metric values and sum them up
    sigma = df_mostSimilar['metric'].abs().sum()

    for id_train in im_data_train.IDs:
        moved_label_path = repo_path / 'data'/'voxelmorph'/f'moved_labels_{id_train}_to_{id_val}.nii.gz'
        moved_label = utils.getArrayfromPath(moved_label_path)
        #get similarity metric value
        simMetric = df_mostSimilar[df_mostSimilar['id_train']==int(id_train)]['metric'].abs().values[0]
        #accumulate per tissue
        for tissue in range(1,4):
            weighted_atlas[tissue] += (moved_label==tissue)*(simMetric/sigma) #wieghting by the similarity metric


    #get argmax of the weighted atlas
    weighted_label = np.argmax(weighted_atlas, axis=0)
    #get gorund truth
    groundT = pat_val.labels(format='np')
    #compute the dice score
    dice = [utils.dice_score(groundT==tissue, weighted_label==tissue) for tissue in range(1,4)]
    #save the dice score in a dataframe. The first column is the id_val, then CSF, GM, WM
    df_dice = pd.DataFrame([[id_val] + dice], columns=['id_val', 'CSF', 'GM', 'WM'])
    #concatenate in df_maxM
    df_weightedAtlas = pd.concat([df_weightedAtlas, df_dice], axis=0)
    #save as csv
    df_weightedAtlas.to_csv(repo_path / 'data'/'results'/f'weighted_labels_dice.csv', index=False)